# Grid Object Examples

The `Grid` object represents a grid structure in the `power_grid_model_ds` library. This documentation provides a comprehensive guide on how to use the `Grid` object, including its initialization, adding and removing nodes and branches, activating and deactivating branches, and creating a grid from a text file.

## Initialization

### Creating an Empty Grid

To create an empty grid, use the `Grid.empty()` method. This method initializes an empty grid with default settings.


In [1]:
from power_grid_model_ds import Grid

grid = Grid.empty()

This creates an empty grid instance. You can verify that the grid is initialized correctly by checking that it contains the fields `arrays` and `graphs`.

## Building a Basic Grid

In this section, we'll build a basic grid step by step, adding each component separately.

### Adding Substations

First, we add a substation to the grid. Substations are critical nodes in the grid that typically connect different voltage levels.


In [2]:
from power_grid_model_ds.arrays import NodeArray
from power_grid_model_ds.enums import NodeType

substation = NodeArray(id=[101], u_rated=[10_500.0], node_type=[NodeType.SUBSTATION_NODE.value])
grid.append(substation, check_max_id=False)
substation = NodeArray(id=[102, 103, 104, 105, 106], u_rated=[10_500.0] * 5, node_type=[NodeType.UNSPECIFIED.value] * 5)
grid.append(substation, check_max_id=False)

### Adding Lines

Next, we add lines to the grid. Lines represent the connections between nodes and can be active or inactive.


In [3]:
from power_grid_model_ds.arrays import LineArray

lines = LineArray(
    id=[201, 202, 203, 204],
    from_status=[1, 1, 0, 1],
    to_status=[1, 1, 0, 1],
    from_node=[101, 102, 103, 101],
    to_node=[102, 103, 104, 105],
    i_n=[200.0] * 4,
    r1=[0.1] * 4,
    x1=[0.03] * 4,
    c1=[0.0] * 4,
    tan1=[0.0] * 4,
)
grid.append(lines, check_max_id=False)

This adds four lines to the grid, connecting different nodes. One of these lines is inactive (`from_status` and `to_status` are 0).

### Adding Transformers

Transformers are added to connect nodes with different voltage levels.


In [4]:
from power_grid_model_ds.arrays import TransformerArray

trafo = TransformerArray.empty(1)
trafo.id = 301
trafo.from_status = 1
trafo.to_status = 1
trafo.from_node = 102
trafo.to_node = 106
grid.append(trafo, check_max_id=False)

This adds a transformer between nodes 102 and 106.

### Adding Links

Links are another type of connection between nodes, often used for specific purposes like bypassing parts of the network.


In [5]:
from power_grid_model_ds.arrays import LinkArray

link = LinkArray.empty(1)
link.id = 601
link.from_status = 1
link.to_status = 1
link.from_node = 104
link.to_node = 105
grid.append(link, check_max_id=False)

This adds a link between nodes 104 and 105.


### Adding Loads

Loads represent consumers of electricity connected to different nodes.


In [6]:
from power_grid_model_ds.arrays import SymLoadArray

loads = SymLoadArray(
    id=[401, 402, 403, 404],
    node=[102, 103, 104, 105],
    type=[1] * 4,
    p_specified=[1_000_000.0] * 4,
    q_specified=[1_000_000.0] * 4,
    status=[1] * 4,
)
grid.append(loads, check_max_id=False)

This adds four loads to the grid, each connected to a different node.

### Adding a Source

Finally, a source is added to provide power to the grid.


In [7]:
from power_grid_model_ds.arrays import SourceArray

source = SourceArray(id=[501], node=[101], status=[1], u_ref=[0.0])
grid.append(source, check_max_id=False)

This adds a power source at node 101.

After adding all these components, you should check the IDs to ensure there are no conflicts.


In [8]:
grid.check_ids()

## Node Operations

### Adding a Node

To add a node to the grid, use the `add_node` method. This method adds a new node to the grid's node array.


In [9]:
from power_grid_model_ds.arrays import NodeArray

new_node = NodeArray.zeros(1)
grid.add_node(node=new_node)

After adding the node, the total number of nodes in the grid increases by one.

### Deleting a Node

To delete a node from the grid, use the `delete_node` method. This method removes the specified node from the grid's node array.


In [10]:
target_node = grid.node.get(101)
grid.delete_node(node=target_node)

## Branch Operations

### Adding a Line

To add a line to the grid, use the `add_branch` method with a `LineArray` object. This method adds a new line between specified nodes.


In [11]:
from power_grid_model_ds.arrays import LineArray

new_line_array = LineArray.zeros(1)
new_line_array.from_node = 102
new_line_array.to_node = 105
grid.add_branch(branch=new_line_array)

After adding the line, the total number of lines in the grid increases by one, and the line is added to the graph structure.

### Deleting a Line

To delete a line from the grid, use the `delete_branch` method with the target line array.


In [12]:
target_line = grid.line.get(202)
grid.delete_branch(branch=target_line)

After deleting the line, the total number of lines in the grid decreases by one, and the line is removed from the graph structure.

### Adding a Link

To add a link to the grid, use the `add_branch` method with a `LinkArray` object. This method adds a new link between specified nodes.


In [13]:
from power_grid_model_ds.arrays import LinkArray

new_link_array = LinkArray.zeros(1)
new_link_array.from_node = 102
new_link_array.to_node = 105
grid.add_branch(branch=new_link_array)

After adding the link, the total number of links in the grid increases by one.

### Adding a Transformer

To add a transformer to the grid, use the `add_branch` method with a `TransformerArray` object. This method adds a new transformer between specified nodes.


In [14]:
from power_grid_model_ds.arrays import TransformerArray

new_transformer_array = TransformerArray.zeros(1)
new_transformer_array.from_node = 102
new_transformer_array.to_node = 105
grid.add_branch(branch=new_transformer_array)

After adding the transformer, the total number of transformers in the grid increases by one.

### Deleting a Transformer

To delete a transformer from the grid, use the `delete_branch` method with the target transformer array.


In [15]:
target_transformer = grid.transformer.get(301)
grid.delete_branch(branch=target_transformer)

After deleting the transformer, the total number of transformers in the grid decreases by one.

## Branch Activation and Deactivation

### Activating a Branch

To activate an inactive branch, use the `make_active` method with the target branch array.


In [16]:
target_line = grid.line.get(203)
grid.make_active(branch=target_line)

After activating the branch, its `from_status` and `to_status` are set to active, and the branch is added to the active graph.

### Deactivating a Branch

To deactivate an active branch, use the `make_inactive` method with the target branch array.


In [17]:
target_line = grid.line.get(203)
grid.make_inactive(branch=target_line)

After deactivating the branch, its `from_status` or `to_status` is set to inactive, and the branch is removed from the active graph.
